In [1]:
# import lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


In [2]:
# to split and preprocess
def split_scalar(indep_X,dep_Y):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    X_train,X_test,Y_train,Y_test = train_test_split(indep_X,dep_Y,test_size =0.30,random_state =0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train,X_test,Y_train,Y_test

In [19]:
# r2 for regression
def r2_score_pred(regressor,X_test,Y_test):
    from sklearn.metrics import r2_score
    test_pred = regressor.predict(X_test)
    r2 = r2_score(Y_test,test_pred)
    return r2

In [20]:
# linear regression
def linear(X_train,X_test,Y_train):
    from sklearn.linear_model import LinearRegression
    regressor = LinearRegression()
    regressor.fit(X_train,Y_train)
    r2 = r2_score_pred(regressor, X_test,Y_test)
    return r2

In [33]:
# svm
def svm(X_train,X_test,Y_train):
    from sklearn.svm import SVR
    regressor = SVR(kernel = 'linear')
    regressor.fit(X_train,Y_train)
    r2 = r2_score_pred(regressor,X_test,Y_test)
    return r2

In [45]:
# decision
def dt(X_train,X_test,Y_train):
    from sklearn.tree import DecisionTreeRegressor
    regressor = DecisionTreeRegressor(criterion = 'squared_error',random_state =0)
    regressor.fit(X_train,Y_train)
    r2 = r2_score_pred(regressor,X_test,Y_test)
    return r2

In [46]:
# random
def rf(X_train,X_test,Y_train):
    from sklearn.ensemble import RandomForestRegressor
    regressor = RandomForestRegressor(n_estimators = 20,random_state =0)
    regressor.fit(X_train,Y_train)
    r2 = r2_score_pred(regressor,X_test,Y_test)
    return r2

In [47]:
def rfefeature(indep_X,dep_Y,n):
    rfelist =[]
    
    from sklearn.linear_model import LinearRegression
    lin = LinearRegression()
    
    from sklearn.svm import SVR
    svrl = SVR(kernel = 'linear')
    
    from sklearn.tree import DecisionTreeRegressor
    dt = DecisionTreeRegressor(criterion = 'squared_error',random_state =0)
    
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor(n_estimators = 20,random_state =0)
    
    rfemodellist = [lin,svrl,dt,rf]
    from sklearn.feature_selection import RFE
    for i in rfemodellist:
        print(i)
        log_rfe = RFE(estimator=i, n_features_to_select=n)
        log_fit = log_rfe.fit(indep_X,dep_Y)
        log_rfe_feature = log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)
    return rfelist


In [73]:
# table
def rfe_regression(r2score_lin,r2score_svml,r2score_dt,r2score_rf):
    df = pd.DataFrame(index = ['Linear','SVM','DecisionTree','RandomForest'],columns = ['Linear','SVM_L','DecisionTree','RandomForest'])
    for number,idex in enumerate(df.index):
        df['Linear'][idex] = acclin[number]
        df['SVM_L'][idex] = accsvml[number]
        df['DecisionTree'][idex] = accdt[number]
        df['RandomForest'][idex] = accrf[number]
    return df

In [74]:
dataset = pd.read_csv('placement.csv')
df2 = dataset

In [75]:
df2.isna().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

In [76]:
df2['salary'].fillna(0,inplace = True)

In [77]:
df2 = pd.get_dummies(df2,drop_first =True)
df2 = df2.astype(int)
# df2

In [78]:
indep_X = df2.drop('salary',axis =1)
dep_Y = df2['salary']

In [79]:
rfelist = rfefeature(indep_X,dep_Y,8)
r2score_lin =[]
r2score_svml =[]
r2score_dt =[]
r2score_rf =[]

LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=20, random_state=0)


In [81]:
for i in rfelist:
    X_train,X_test,Y_train,Y_test = split_scalar(i,dep_Y)
    
    r2_lin = linear(X_train,X_test,Y_train)
    r2score_lin.append(r2_lin)
    
    r2_svml = svm(X_train,X_test,Y_train)
    r2score_svml.append(r2_svml)
    
    r2_dt = dt(X_train,X_test,Y_train)
    r2score_dt.append(r2_dt)
    
    r2_rf = rf(X_train,X_test,Y_train)
    r2score_rf.append(r2_rf)
    

In [82]:
result  = rfe_regression(acclin,accsvml,accdt,accrf)
result#4

,Linear,SVM_L,DecisionTree,RandomForest
Linear,0.820778,-0.181052,0.780289,0.769283
SVM,0.818303,-0.179371,-0.36938,0.681221
DecisionTree,0.826751,-0.179703,0.162068,0.70175
RandomForest,0.821699,-0.179793,0.740366,0.746775


In [69]:
result  = rfe_regression(acclin,accsvml,accdt,accrf)
result#6

,Linear,SVM_L,DecisionTree,RandomForest
Linear,0.852903,-0.181273,0.834328,0.823015
SVM,0.845284,-0.179707,0.285856,0.730111
DecisionTree,0.816023,-0.180008,-0.264951,0.685023
RandomForest,0.810046,-0.179766,0.18431,0.700754


In [72]:
result  = rfe_regression(acclin,accsvml,accdt,accrf)
result#8

,Linear,SVM_L,DecisionTree,RandomForest
Linear,0.820778,-0.181052,0.780289,0.769283
SVM,0.818303,-0.179371,-0.36938,0.681221
DecisionTree,0.826751,-0.179703,0.162068,0.70175
RandomForest,0.821699,-0.179793,0.740366,0.746775
